In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))

import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'  # For sharper figures, but it takes more time
from tqdm import tqdm
import scipy as sp
from copy import deepcopy 

# LISA tools
from lisatools.utils.constants import *
from lisatools.sensitivity  import AE1SensitivityMatrix

# BBHX: Suppress the print statements from the BBHX module like: "No CuPy or GPU PhenomHM module"
# This is done to avoid cluttering the output when running sp.optimize.differential_evolution
from contextlib import contextmanager

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

with suppress_stdout():
    from bbhx.waveformbuild import BBHWaveformFD

# My modules
from tools.LISASimulator import LISASimulator
from tools.time_freq_likelihood import TimeFreqLikelihood
from tools.likelihood import get_dh, get_hh, TimeFreqSNR
import tools.likelihood as likelihood
from tools.MBHB_differential_evolution import MBHB_finder

In [2]:
from multiprocessing import cpu_count
print(cpu_count())

11


In [3]:
Tobs = YRSID_SI/12
dt = 5.
include_T_channel = False # Set to True if you want to include the T channel in the simulation, otherwise only A and E channels will be included.

wave_gen = BBHWaveformFD(amp_phase_kwargs=dict(run_phenomd=False))
sim = LISASimulator(Tobs=Tobs, dt=dt, wave_gen=wave_gen, include_T_channel=include_T_channel)

m1 = 3e5
m2 = 1.5e5
a1 = 0.2
a2 = 0.4
dist = 4 * PC_SI * 1e9  # distance in Gpc
phi_ref = np.pi/2
f_ref = 0.0
inc = np.pi/3
lam = np.pi/1.
beta = np.pi/4.
psi = np.pi/4.
t_ref = 0.95 * Tobs
#t_ref = round(0.9 * Tobs / dt) * dt  # round to the nearest multiple of dt, to force t_ref to be a part of t_array

parameters = np.array([m1, m2, a1, a2, dist, phi_ref, f_ref, inc, lam, beta, psi, t_ref])

modes = [(2,2), (2,1), (3,3), (3,2), (4,4), (4,3)]
waveform_kwargs = dict(length=1024, direct=False, fill=True, squeeze=False, modes=modes)

data_t, data_f, f_array, t_array, sens_mat = sim(seed = 42, parameters=parameters, waveform_kwargs=waveform_kwargs)
waveform_kwargs.update(freqs=f_array)

print(sim.SNR_optimal()[0])

3595.48440646233


## Pre-Merger

In [4]:
# Not doing the premerger now to see if the code works better with the merger included
time_before_merger = 60*60
cutoff_time = t_ref - time_before_merger
max_time = t_ref + 60*60*12

def pre_merger(gravitational_wave_data_t, time_before_merger, t_ref, t_array):
        cutoff_time = t_ref - time_before_merger
        cutoff_index = np.searchsorted(t_array, cutoff_time)
        data_t_truncated = gravitational_wave_data_t[:, :cutoff_index]
        return data_t_truncated, cutoff_index

data_t_truncated, cutoff_index =  pre_merger(sim.signal_t[0], time_before_merger, t_ref, t_array)


# The SNR does not depend on the distance. 
Change the guess_distance to see that the SNR is the same. The distance is calculated based on the amplitude.
- This only works when all the other parameters in template are set to the real values. 
- The hope is to use differential_evolution to calculate the remaining parameters reasonably well and get an estimate of distance from that

In [5]:
guess_distance = dist * 1000
signal_with_which_to_test_this = sim.signal_f[0] #data_f
template = wave_gen(
    m1,
    m2,
    a1,
    a2,
    guess_distance, 
    phi_ref,
    f_ref, 
    inc,
    lam,
    beta,
    psi,
    t_ref,
    **waveform_kwargs
)
template = template[0, :2]
print(likelihood.template_snr(signal_with_which_to_test_this, template, AE1SensitivityMatrix(f_array), df=sim.df))

hh = get_hh(template, AE1SensitivityMatrix(f_array), df=sim.df)
dh = get_dh(signal_with_which_to_test_this, template, AE1SensitivityMatrix(f_array), df=sim.df)
amplitude = dh/hh
new_distance = guess_distance /  amplitude

print( "Percentage diff     = ", (new_distance-dist)*100/dist , "%" )
print( "True distance       = ",  dist/(PC_SI*1e9), "Gpc")
print( "Dist from Amplitude = ",  new_distance/(PC_SI*1e9), "Gpc")
print( "SNR calculated      = " , dh/np.sqrt(hh))

3595.484406462329
Percentage diff     =  4.1757123185153e-14 %
True distance       =  4.0 Gpc
Dist from Amplitude =  4.000000000000002 Gpc
SNR calculated      =  3595.484406462329


The SNR also does not change with the time-frequency SNR with pre-merger. Change guess_distance and see that nothing changes.
- Again, this only works when all the other parameters in template are set to the real values. 
- The hope is to use differential_evolution to calculate the remaining parameters reasonably well and get an estimate of distance from that.
- But it's good that using time-freq SNR also works to get the distance.
- Change the value of, say, m1 drastically and see that the distance estimate from the equation is awful, which makes sense.

In [6]:
guess_distance = dist 
parameters_new = [
    m1*(1),  # Slightly change m1 to see the effect
    m2,
    a1*(1),  # Slightly change a1 to see the effect
    a2,
    guess_distance,
    phi_ref,
    f_ref,
    inc*(1 ),
    lam,
    beta,
    psi,
    t_ref
]
analysis = TimeFreqSNR(
    data_t_truncated,
    wave_gen=wave_gen,
    nperseg=5000,
    dt_full=dt,
    cutoff_index=cutoff_index,
    pre_merger=True
)
analysis.get_stft_of_data()
SNR, amplitude = analysis.calculate_time_frequency_SNR(*parameters_new, waveform_kwargs=waveform_kwargs)
new_distance = guess_distance /  amplitude
print((new_distance - dist)/(PC_SI*1e9) , (new_distance-dist)/dist)
print( "True distance       = ",  dist/(PC_SI*1e9), "Gpc")
print( "Dist from Amplitude = ",  new_distance/(PC_SI*1e9), "Gpc")
print( "SNR calculated      = ",  SNR)

0.0 0.0
True distance       =  4.0 Gpc
Dist from Amplitude =  4.0 Gpc
SNR calculated      =  0.7273685958383449


# Differential Evolution Analysis

In [7]:
boundaries = {}
boundaries['Total_Mass'] = [1e5, 6e5]
boundaries['Mass_Ratio'] = [0.05, 0.999999]
boundaries['Spin1'] = [-1, 1]
boundaries['Spin2'] = [-1, 1]
boundaries['Distance'] = [1e3, 10e3] # in Mpc i.e. dL / (PC_SI * 1e6)
boundaries['Phase'] = [0.0, 2 * np.pi]
boundaries['cos(Inclination)'] = [-1, 1]
boundaries['Ecliptic_Longitude'] = [0, 2*np.pi]
boundaries['sin(Ecliptic_Latitude)'] = [-1, 1]
boundaries['Polarization'] = [0, np.pi]
boundaries['Coalescence_Time'] = [cutoff_time, max_time]  

In [8]:
DifferentialEvolution = MBHB_finder(
    data_t = sim.signal_t[0],           # For pre-merger, use data_t_truncated. For full signal, use sim.signal_t[0]
    wave_gen= wave_gen,
    waveform_kwargs=waveform_kwargs,
    boundaries=boundaries,
    nperseg=5000,
    dt_full= dt,
    pre_merger=False,
    #cutoff_index=cutoff_index,
    true_parameters=parameters,
)
DifferentialEvolution.get_stft_of_data()

In [13]:
differential_evolution_kwargs = {
    'strategy': 'best1exp',
    'popsize': 3,
    'tol': 1e-7,
    'maxiter': 40,
    'recombination': 1,
    'mutation': (0.7, 1),
    'polish': False,
    'disp': True,
    'workers': -1,  # Use all available CPU cores
    'updating': 'deferred',
}

fixed_parameters = {
    'Total_Mass': m1 + m2,
    'Mass_Ratio': m2 / m1,
    'Spin1': a1,
    'Spin2': a2,
    'Distance': boundaries['Distance'][0] + 0.5 * (boundaries['Distance'][1] - boundaries['Distance'][0]), # Always include this in fixed parameters
    'Phase': phi_ref,
    'cos(Inclination)': np.cos(inc),
    #'Ecliptic_Longitude': lam,
    'sin(Ecliptic_Latitude)': np.sin(beta),
    'Polarization': psi,
    'Coalescence_Time': t_ref,
}

In [14]:
found_parameters, found_snr_found, results = DifferentialEvolution.find_MBHB(differential_evolution_kwargs=differential_evolution_kwargs, fixed_parameters=fixed_parameters, number_of_searches=2)
# The number of messages like "No CuPy or GPU PhenomHM module" you see is exactly equal to the number of workers used. On this macbook, it has 11 cores, so there are 11 messages. Imma just live with this

time SNR  0.28
initial guess 0.00766716189464721
No CuPy
No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.
No CuPy or GPU response available.
No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.
No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.
No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.
No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.
No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.
No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.
No CuPy
No CuPy or GPU PhenomHM

In [15]:
print(DifferentialEvolution)

Index Parameter                 Lower Bound    Found 1             Found 2             True                 Upper Bound     Status    
--------------------------------------------------------------------------------------------------------------------------------------
0     Total_Mass                100000         450000              450000              450000               600000          fixed     
1     Mass_Ratio                0.05           0.5                 0.5                 0.5                  0.999999        fixed     
2     Spin1                     -1             0.2                 0.2                 0.2                  1               fixed     
3     Spin2                     -1             0.4                 0.4                 0.4                  1               fixed     
4     Distance                  1000           3999.45             5744.73             4000                 10000           variable  
5     Phase                     0              1.5708  

- The reason why the distance estimate can be so bad (sometimes even outside the priors) is because it is calculated from the amplitude that is calculated with a template generated by the other found parameters
- The other found parameters don't have to be close to the true parameters, unless you run it for longer
- With only 1 variable parameter, it works properly

In [12]:
"""
def transform_parameters(x):
    all_parameters = np.zeros(11)
    all_parameters[0] = x[0] + x[1]
    all_parameters[1] = x[1] / x[0]
    all_parameters[2] = x[2]
    all_parameters[3] = x[3]
    all_parameters[4] = x[4] / (PC_SI * 1e6)
    all_parameters[5] = x[5]
    all_parameters[6] = np.cos(x[7])
    all_parameters[7] = x[8]
    all_parameters[8] = np.sin(x[9])
    all_parameters[9] = x[10]
    all_parameters[10] = x[11]
    return all_parameters
"""
"""print(f"{'Index':<5} {'Parameter':<25} {'Lower Bound':<15} {'Found':<20} {'True':<20} {'Upper Bound':<15} {'Status':<10}")
print('-' * 120)

for i, (param, bounds) in enumerate(boundaries.items()):
    lower, upper = bounds
    found = found_parameters[i]
    true = transform_parameters(parameters)[i]

    # Determine if the parameter was fixed or optimized
    if param == 'Distance':
        status = 'variable'  # Special case: always print Distance as variable
    elif param in fixed_parameters:
        status = 'fixed'
    else:
        status = 'variable'

    print(f"{i:<5} {param:<25} {lower:<15.6g} {found:<20.6g} {true:<20.6g} {upper:<15.6g} {status:<10}")
"""

'print(f"{\'Index\':<5} {\'Parameter\':<25} {\'Lower Bound\':<15} {\'Found\':<20} {\'True\':<20} {\'Upper Bound\':<15} {\'Status\':<10}")\nprint(\'-\' * 120)\n\nfor i, (param, bounds) in enumerate(boundaries.items()):\n    lower, upper = bounds\n    found = found_parameters[i]\n    true = transform_parameters(parameters)[i]\n\n    # Determine if the parameter was fixed or optimized\n    if param == \'Distance\':\n        status = \'variable\'  # Special case: always print Distance as variable\n    elif param in fixed_parameters:\n        status = \'fixed\'\n    else:\n        status = \'variable\'\n\n    print(f"{i:<5} {param:<25} {lower:<15.6g} {found:<20.6g} {true:<20.6g} {upper:<15.6g} {status:<10}")\n'